# 语言模型Transformer库-Datasets组件实践

## 一、加载和保存

### 1、加载数据集

In [ ]:
from datasets import load_dataset
datasets = load_dataset("madao33/new-title-chinese")
datasets

### 2、加载数据集任务

In [ ]:
boolq_dataset = load_dataset("super_glue", "boolq")
boolq_dataset

### 3、加载划分数据集

In [ ]:
# 使用train关键字加载训练数据集
dataset = load_dataset("madao33/new-title-chinese", split="train")
dataset

In [ ]:
# 按区间加载训练数据集
dataset = load_dataset("madao33/new-title-chinese", split="train[10:100]")
dataset

In [ ]:
# 按百分比加载数据集
dataset = load_dataset("madao33/new-title-chinese", split="train[:50%]")
dataset

In [ ]:
# 加载名为"madao33/new-title-chinese"的数据集，并将其划分为训练集的前50%和后50%
datasets = load_dataset("madao33/new-title-chinese", split=["train[:50%]", "train[50%:]"])
datasets

### 4、加载本地数据集

In [ ]:
from datasets import load_from_disk

# 从当前目录下的"./processed_data"文件夹中加载数据集，并将其赋值给变量processed_datasets。
processed_datasets = load_from_disk("./processed_data")
processed_datasets

In [ ]:
# data_files参数指定了数据集的文件路径，
# split参数指定了要加载的数据集的划分方式（这里是"train"）。
dataset = load_dataset("csv", data_files="/root/pretrains/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv", split="train")
dataset

In [ ]:
from datasets import Dataset

dataset = Dataset.from_csv("/root/pretrains/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv")
dataset

In [ ]:
import pandas as pd

data = pd.read_csv("/root/pretrains/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv")
dataset = Dataset.from_pandas(data)
dataset

In [ ]:
# 自定义List加载数据集
# List格式的数据需要内嵌{}，明确数据字段
data = [{"text": "abc"}, {"text": "def"}]
# data = ["abc", "def"]
Dataset.from_list(data)

In [ ]:
# 使用load_dataset函数从指定的JSON文件中加载数据集，并将其赋值给变量dataset。
# 其中，field参数指定了要加载的数据集的字段名（这里是"data"）
load_dataset("json", data_files="./cmrc2018_trial.json", field="data")

In [ ]:
# 使用自定义的加载脚本load_script.py加载数据集，并将其赋值给变量dataset。
# 其中，split参数指定了要加载的数据集的划分方式（这里是"train"）
dataset = load_dataset("./load_script.py", split="train")
dataset

### 5、保存数据集

In [ ]:
# 将处理后的数据集datasets保存到当前目录下的"./processed_data"文件夹中。
datasets.save_to_disk("./processed_data")

## 二、查看数据集

In [ ]:
datasets = load_dataset("madao33/new-title-chinese")
# 查看索引为1的数据
datasets["train"][1]
# 查看数据集前2条数据
datasets["train"][:2]
# 获取训练集中标题列的前5个元素
datasets["train"]["title"][:5]
# 获取训练集的列名
datasets["train"].column_names
# 获取训练集中的特征信息
datasets["train"].features

## 三、划分数据集

In [ ]:
dataset = datasets["train"]
# 使用train_test_split方法将数据集划分为训练集和测试集，其中测试集占比为0.1
dataset.train_test_split(test_size=0.1)

In [ ]:
# 使用train_test_split方法将数据集划分为训练集和测试集，
# 其中测试集占比为0.1，并按照"label"列进行分层抽样
# 保证训练集和测试集中各类别的样本比例与原始数据集中的比例相同
dataset.train_test_split(test_size=0.1, stratify_by_column="label")

## 四、处理数据集

In [ ]:
# 选取：选择训练集中索引为0和1的数据
datasets["train"].select([0, 1])

# 过滤: 过滤训练集中标题包含"中国"的数据，获取过滤后的训练集的前5个标题
filter_dataset = datasets["train"].filter(lambda example: "中国" in example["title"])
filter_dataset["title"][:5]

# 映射: 通过函数进行数据集过滤
def add_prefix(example):
    example["title"] = 'Prefix: ' + example["title"]
    return example
prefix_dataset = datasets.map(add_prefix)
prefix_dataset["train"][:10]["title"]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
def preprocess_function(example):
    model_inputs = tokenizer(example["content"], max_length=512, truncation=True)
    labels = tokenizer(example["title"], max_length=32, truncation=True)
    # labels就是title编码的结果
    # "labels"键是用来存储标题编码结果的。在训练模型时，我们通常需要指定输入的标签或目标，以便计算损失函数并进行模型优化
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
processed_datasets = datasets.map(preprocess_function)
processed_datasets

In [ ]:
# 使用了4个进程并行地对数据集进行处理，以提高处理速度
processed_datasets = datasets.map(preprocess_function, num_proc=4)
processed_datasets

In [ ]:
# 使用了批处理的方式对数据集进行处理，可以处理较大的数据集
processed_datasets = datasets.map(preprocess_function, batched=True)
processed_datasets

In [ ]:
# 使用批处理的方式对数据集进行处理，并且移除了原始数据集中的列信息，只保留了处理后的数据集。
processed_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)
processed_datasets

## 五、DataCollator
DataCollatorWithPadding是Hugging Face Transformers库中的一个类，用于处理文本数据。它的主要功能和作用如下：

- 填充（Padding）: DataCollatorWithPadding可以将不同长度的文本序列进行填充，使它们具有相同的长度。这对于批量输入到神经网络模型中非常重要，因为模型通常要求输入数据具有相同的形状。
- 动态填充（Dynamic Padding）: DataCollatorWithPadding可以根据输入数据的最大长度动态地调整填充的长度。这意味着如果一个批次中的最长文本序列长度为100，那么所有其他较短的文本序列将被填充至长度为100。
- 指定填充值（Padding Value）: DataCollatorWithPadding允许用户指定填充值，默认为0。这可以用于区分填充部分和实际文本内容。
- 支持多种框架（Framework Support）: DataCollatorWithPadding支持多种深度学习框架，包括PyTorch、TensorFlow等。这使得用户可以在不同的框架之间灵活切换，而无需更改数据处理代码。<br>
DataCollatorWithPadding是一个功能强大的工具，可以帮助用户在处理文本数据时进行填充操作，确保输入数据的一致性和模型训练的稳定性。

### 1、加载数据集

In [ ]:
from transformers import  DataCollatorWithPadding

dataset = load_dataset("csv", data_files="/root/pretrains/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv", split='train')
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset


### 2、数据处理

In [ ]:
def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples
tokenized_dataset = dataset.map(process_function, batched=True, remove_columns=dataset.column_names)
tokenized_dataset

### 3、数据填充

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)
collator

In [ ]:
from torch.utils.data import DataLoader

# 创建DataLoader对象，指定批量大小为4，使用collator进行数据填充，并进行随机打乱
dl = DataLoader(tokenized_dataset, batch_size=4, collate_fn=collator, shuffle=True)
#获取下一个批次的数据
next(enumerate(dl))